## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [2]:
import pandas as pd

df = pd.read_csv('client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [3]:
# View the column names in the data
print("Column Names:", df.columns.tolist())

Column Names: ['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id', 'order_date', 'order_week', 'order_year', 'item_id', 'category', 'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty', 'line_number']


In [4]:
# Use the describe function to gather some basic statistics
description = df.describe()
print("Basic Statistics:\n", description)

Basic Statistics:
           client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number  
count  54639.000000  54639.000000  5.463900e+04  54639.000000  
mean      99.446073      5.004116  5.702646e+02      2.979667  
std      133.164267      5.326599  1.879552e+04      2.436320  
min        0.

In [5]:
# Use this space to do any additional research
# and familiarize yourself with the data.
# Top 3 item categories by entries
top_categories = df['category'].value_counts().head(3)
print("Top 3 Categories by Entries:\n", top_categories)


Top 3 Categories by Entries:
 category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [6]:
# What three item categories had the most entries?
print("Top 3 Categories by Entries:\n", top_categories)

Top 3 Categories by Entries:
 category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [7]:
# For the category with the most entries, which subcategory had the most entries?
top_category = top_categories.index[0]
top_subcategory = df[df['category'] == top_category]['subcategory'].value_counts().head(1)
print(f"Subcategory with most entries in '{top_category}':\n", top_subcategory)

Subcategory with most entries in 'consumables':
 subcategory
bathroom supplies    6424
Name: count, dtype: int64


In [8]:
# Which five clients had the most entries in the data?
top_clients = df['client_id'].value_counts().head(5)
top_clients_list = top_clients.index.tolist()
print("Top 5 Clients by Entries:\n", top_clients)
print("Top 5 Client IDs:", top_clients_list)

Top 5 Clients by Entries:
 client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64
Top 5 Client IDs: [33615, 66037, 46820, 38378, 24741]


In [9]:
# Store the client ids of those top 5 clients in a list.

print("Top 5 Client IDs:", top_clients_list)


Top 5 Client IDs: [33615, 66037, 46820, 38378, 24741]


In [10]:
# How many total units (the qty column) did the client with the most entries order order?
top_client_units = df[df['client_id'] == top_clients_list[0]]['qty'].sum()
print("Total Units Ordered by Top Client:", top_client_units)

Total Units Ordered by Top Client: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [11]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
# Calculate the line subtotal
df['line_subtotal'] = df['unit_price'] * df['qty']

# Print the 'unit_price' column to verify
print(df['unit_price'])


0        1096.80
1          24.95
2          13.52
3          36.42
4         195.10
          ...   
54634      83.13
54635     206.59
54636      65.66
54637       1.48
54638       3.01
Name: unit_price, Length: 54639, dtype: float64


In [12]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
def calculate_shipping(total_weight):
    if total_weight > 50:
        return total_weight * 7
    return total_weight * 10

df['total_weight'] = df['unit_weight'] * df['qty']
df['shipping_price'] = df["total_weight"].apply(calculate_shipping)

# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']].head(3)


,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64


In [21]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df['line_price'] = round( (df['line_subtotal'] + df['shipping_price']) * 1.0925,2)
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.026250,109651.50
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.468625,211.05
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.937600,68.64
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.221400,699.48
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.803500,-2598.20


In [22]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df['line_cost'] = df['unit_cost'] * df['qty'] + df['shipping_price']


In [15]:
# Create a column for the profit of each line using line cost and line price
df['line_profit'] = df['line_price'] - df['line_cost']



## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [16]:
# Check your work using the totals above
receipts = {
    2742071: 152811.89,
    2173913: 162388.71,
    6128929: 923441.25
}
for order_id, expected_total in receipts.items():
    calculated_total = df[df['order_id'] == order_id]['line_price'].sum()
    print(f"Order {order_id} total: ${calculated_total:.2f} (Expected: ${expected_total})")


Order 2742071 total: $152811.90 (Expected: $152811.89)
Order 2173913 total: $162388.72 (Expected: $162388.71)
Order 6128929 total: $923441.24 (Expected: $923441.25)


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [23]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_clients_summary = df[df['client_id'].isin(top_clients_list)].groupby('client_id').agg({
    'qty': 'sum',
    'shipping_price': 'sum',
    'line_price': 'sum',
    'line_cost': 'sum',
    'line_profit': 'sum'
}).reset_index()
top_clients_summary 


,client_id,qty,shipping_price,line_price,line_cost,line_profit
0,24741,239862,5126448.37,82268892.02,45688899.71,3.657999e+07
1,33615,64313,1828984.89,8377308.52,6175313.91,2.201995e+06
2,38378,73667,3429455.40,12906550.87,9634720.98,3.271830e+06
3,46820,75768,1601448.84,9743794.36,7007191.64,2.736603e+06
4,66037,43018,1395151.85,10259514.79,7004482.98,3.255032e+06


In [18]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
# Step 1: Calculate necessary columns
df['line_subtotal'] = df['unit_price'] * df['qty']  # Total revenue for each line
df['profit'] = df['line_subtotal'] - df['shipping_price']  # Example profit calculation

# Step 2: Identify the top 5 clients with the most entries
top_clients = df['client_id'].value_counts().head(5).index.tolist()

# Step 3: Filter the DataFrame for the top 5 clients
top_clients_df = df[df['client_id'].isin(top_clients)]

# Step 4: Group by client_id and calculate summary metrics
summary_df = top_clients_df.groupby('client_id').agg(
    total_units_purchased=('qty', 'sum'),
    total_shipping_price=('shipping_price', 'sum'),
    total_revenue=('line_subtotal', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

# Step 5: Define a function to convert dollar amounts to millions
def convert_to_millions(amount):
    return amount / 1_000_000

# Apply the function to format monetary values
summary_df['total_shipping_price'] = summary_df['total_shipping_price'].apply(convert_to_millions)
summary_df['total_revenue'] = summary_df['total_revenue'].apply(convert_to_millions)
summary_df['total_profit'] = summary_df['total_profit'].apply(convert_to_millions)

# Step 6: Display the summary DataFrame
print(summary_df)

   client_id  total_units_purchased  total_shipping_price  total_revenue  \
0      24741                 239862              5.126448      70.176885   
1      33615                  64313              1.828985       5.839032   
2      38378                  73667              3.429455       8.384321   
3      46820                  75768              1.601449       7.317356   
4      66037                  43018              1.395152       7.995708   

   total_profit  
0     65.050437  
1      4.010047  
2      4.954866  
3      5.715907  
4      6.600557  


In [19]:
# Format the data and rename the columns to names suitable for presentation.
# Rename the columns to reflect the change in the money format. 
# Define the money columns
money_columns = ['total_shipping_price', 'total_revenue', 'total_profit']

# Define a function to format a dollar amount in millions
def currency_format_millions(amount):
    return f"${amount:.2f}M"

# Apply the currency_format_millions function to the money columns
for column in money_columns:
    summary_df[column] = summary_df[column].apply(currency_format_millions)

# Rename the columns for presentation
summary_df.rename(columns={
    'total_units_purchased': 'Total Units Purchased',
    'total_shipping_price': 'Total Shipping Price (in Millions)',
    'total_revenue': 'Total Revenue (in Millions)',
    'total_profit': 'Total Profit (in Millions)'
}, inplace=True)

# Display the formatted DataFrame
print(summary_df)

   client_id  Total Units Purchased Total Shipping Price (in Millions)  \
0      24741                 239862                             $5.13M   
1      33615                  64313                             $1.83M   
2      38378                  73667                             $3.43M   
3      46820                  75768                             $1.60M   
4      66037                  43018                             $1.40M   

  Total Revenue (in Millions) Total Profit (in Millions)  
0                     $70.18M                    $65.05M  
1                      $5.84M                     $4.01M  
2                      $8.38M                     $4.95M  
3                      $7.32M                     $5.72M  
4                      $8.00M                     $6.60M  


In [20]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
# Sort the DataFrame by "Total Profit (in Millions)" in descending order
sorted_summary_df = summary_df.sort_values(
    by='Total Profit (in Millions)', 
    ascending=False
)

# Display the sorted DataFrame
print(sorted_summary_df)

   client_id  Total Units Purchased Total Shipping Price (in Millions)  \
0      24741                 239862                             $5.13M   
4      66037                  43018                             $1.40M   
3      46820                  75768                             $1.60M   
2      38378                  73667                             $3.43M   
1      33615                  64313                             $1.83M   

  Total Revenue (in Millions) Total Profit (in Millions)  
0                     $70.18M                    $65.05M  
4                      $8.00M                     $6.60M  
3                      $7.32M                     $5.72M  
2                      $8.38M                     $4.95M  
1                      $5.84M                     $4.01M  
